In [12]:
import numpy as np
import json
import requests
import pandas as pd
import geopy
from geopy.distance import geodesic


In [13]:
df = pd.read_csv('resale_data.csv')

print(df.shape)
print(df.info())
df.head(2)

(155821, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155821 entries, 0 to 155820
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   month                           155821 non-null  object 
 1   town                            155821 non-null  object 
 2   public_housing_flat_type        155821 non-null  object 
 3   block                           155821 non-null  object 
 4   street_name                     155821 non-null  object 
 5   storey_range                    155821 non-null  object 
 6   floor_area_sqm                  155821 non-null  float64
 7   flat_model                      155821 non-null  object 
 8   lease_commence_date             155821 non-null  int64  
 9   remaining_lease                 155821 non-null  object 
 10  resale_price_Singapore_dollars  155821 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 13.1+ MB
None


,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_Singapore_dollars
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0


In [14]:

from concurrent.futures import ThreadPoolExecutor

def process_address(add):
    url = 'https://www.onemap.gov.sg/api/common/elastic/search?searchVal=' + add + '&returnGeom=Y&getAddrDetails=Y&pageNum=1'
    print(url)

    response = requests.get(url)
    try:
        data = json.loads(response.text)
    except ValueError:
        print('JSONDecodeError')
        return pd.DataFrame()

    temp_df = pd.DataFrame.from_dict(data["results"])
    temp_df["Name"] = add
    return temp_df

def find_postal(lst, filename, batch_size=50):
    result_list = []

    with ThreadPoolExecutor() as executor:
        for i in range(0, len(lst), batch_size):
            batch_addresses = lst[i:i + batch_size]
            batch_results = list(executor.map(process_address, batch_addresses))
            batch_df = pd.concat(batch_results, ignore_index=True)
            result_list.append(batch_df)

    final_df = pd.concat(result_list, ignore_index=True)
    final_df.to_csv(filename + '.csv')


In [15]:
school = pd.read_csv('school_names.csv',encoding='cp1252')
school.head(10)

,Name,town
0,Admiralty Primary School,Woodlands
1,Ahmad Ibrahim Primary School,Yishun
2,Ai Tong School,Bishan
3,Alexandra Primary School,Bukit Merah
4,Anchor Green Primary School,Sengkang
5,Anderson Primary School,Ang Mo Kio
6,Anglo-Chinese School (Junior),Central
7,Anglo-Chinese School (Primary),Bukit Timah
8,Angsana Primary School,Tampines
9,Ang Mo Kio Primary School,Ang Mo Kio


In [16]:
school_name = list(school['Name'])
unique_school_name = list(set(school_name))

print('Unique addresses:', len(unique_school_name))

Unique addresses: 330


In [17]:
def clean_address(string, dictionary):
    for item in string.split(" "):
        if item in dictionary.keys():
             string = string.replace(item,dictionary[item])
        else:
            string
    return string

In [ ]:
find_postal(unique_school_name, 'school_coordinates.csv', batch_size=50)

In [18]:
import numpy as np
import pandas as pd
df['Year'] = df['month'].str[:4]
df['mth'] = df['month'].str[5:]
df['address'] = df['block'] + ' ' + df['street_name']
display(df)
def clean_address(string, dictionary):
    for item in string.split(" "):
        if item in dictionary.keys():
             string = string.replace(item, dictionary[item])
        else:
            string
    return string

# Load HDB data
hdb = pd.read_csv("hdb_address_geoloc.csv")

# Remove repeated items in the geolocation CSV file
hdb['block'] = hdb['address'].str.split().str.get(0)
hdb['true_add'] = np.where(hdb['BLK_NO'] == hdb['block'], 'yes', 'no')
hdb1 = hdb[hdb['true_add'] == 'yes']
hdbfinal = hdb1.drop_duplicates(subset=['address'], keep='first')
hdbfinal.to_csv('hdb_address_geoloc_final.csv')



# Define text replace dictionary
text_replace_dict = {"ST.": "ST"}

# Create a new column with cleaned addresses
df["address_clean"] = [clean_address(val, text_replace_dict) for val in df["address"]]
df['address_clean'] = np.where(df['address_clean'] == "4C ST GEORGE'S LANE", "4C ST. GEORGE'S LANE", df['address_clean'])

# Load the geolocation data
df_geopts = pd.read_csv('hdb_address_geoloc_final.csv')
df_geopts = df_geopts[['address', 'LATITUDE', 'LONGITUDE', 'POSTAL']]

# Merge DataFrames on the cleaned address column
df_postal = pd.merge(df, df_geopts, how='left', left_on=['address_clean'], right_on=['address'])
df_postal.to_csv('resale_flat_with_postal.csv', index=False)

df_postal.head(3)


,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_Singapore_dollars,Year,mth,address
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017,01,406 ANG MO KIO AVE 10
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017,01,108 ANG MO KIO AVE 4
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017,01,602 ANG MO KIO AVE 5
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0,2017,01,465 ANG MO KIO AVE 10
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0,2017,01,601 ANG MO KIO AVE 5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155816,2023-06,YISHUN,5 ROOM,511B,YISHUN ST 51,10 TO 12,113.0,Improved,2017,93 years 03 months,645000.0,2023,06,511B YISHUN ST 51
155817,2023-06,YISHUN,5 ROOM,513A,YISHUN ST 51,07 TO 09,120.0,3Gen,2018,93 years 09 months,685000.0,2023,06,513A YISHUN ST 51
155818,2023-06,YISHUN,5 ROOM,613,YISHUN ST 61,01 TO 03,121.0,Improved,1987,62 years 11 months,565000.0,2023,06,613 YISHUN ST 61
155819,2023-06,YISHUN,EXECUTIVE,791,YISHUN AVE 2,04 TO 06,143.0,Apartment,1987,63 years 04 months,845000.0,2023,06,791 YISHUN AVE 2


,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price_Singapore_dollars,Year,mth,address_x,address_clean,address_y,LATITUDE,LONGITUDE,POSTAL
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0,2017,01,406 ANG MO KIO AVE 10,406 ANG MO KIO AVE 10,406 ANG MO KIO AVE 10,1.362005,103.853880,560406.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0,2017,01,108 ANG MO KIO AVE 4,108 ANG MO KIO AVE 4,108 ANG MO KIO AVE 4,1.370966,103.838202,560108.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0,2017,01,602 ANG MO KIO AVE 5,602 ANG MO KIO AVE 5,602 ANG MO KIO AVE 5,1.380709,103.835368,560602.0


In [19]:
resale_postal = pd.read_csv("resale_flat_with_postal.csv")
resale_postal1 = resale_postal[['address_clean', 'LATITUDE', 'LONGITUDE']]
display(resale_postal1)

,address_clean,LATITUDE,LONGITUDE
0,406 ANG MO KIO AVE 10,1.362005,103.853880
1,108 ANG MO KIO AVE 4,1.370966,103.838202
2,602 ANG MO KIO AVE 5,1.380709,103.835368
3,465 ANG MO KIO AVE 10,1.366201,103.857201
4,601 ANG MO KIO AVE 5,1.381041,103.835132
...,...,...,...
155816,511B YISHUN ST 51,1.415600,103.842414
155817,513A YISHUN ST 51,1.416040,103.843489
155818,613 YISHUN ST 61,1.420067,103.835284
155819,791 YISHUN AVE 2,1.420520,103.833564


In [20]:
def find_nearest(house, amenity):
    results = {}
    total_houses = len(house)
    for index, flat in enumerate(house.iloc[:, 0]):
        flat_loc = (house.iloc[index, 1], house.iloc[index, 2])

        flat_amenity = ['', '', 100]

        for ind, eachloc in enumerate(amenity.iloc[:, 0]):
            amenity_loc = (amenity.iloc[ind, 1], amenity.iloc[ind, 2])

            # Check for NaN values before calculating distance
            if np.isnan(flat_loc[0]) or np.isnan(flat_loc[1]) or np.isnan(amenity_loc[0]) or np.isnan(amenity_loc[1]):
                continue

            distance = geodesic(flat_loc, amenity_loc)

            if distance < flat_amenity[2]:
                flat_amenity[0] = flat
                flat_amenity[1] = eachloc
                flat_amenity[2] = distance

        results[flat] = flat_amenity
         # Print progress statement
        if (index + 1) % 1000 == 0 or index + 1 == total_houses:
            print(f"Processed {index + 1}/{total_houses} houses")
    return results

In [21]:
def save_file(data, filename):
    data1 = pd.DataFrame(data)
    data2 = data1.transpose()
    data2.to_csv(filename + '.csv', index=False)

In [22]:
df_school_geoloc = pd.read_csv("school_coordinates.csv.csv")
display(df_school_geoloc)

df_school_geoloc.rename(columns={'address':'Name'}, inplace=True)
display(df_school_geoloc)
df_school_geoloc_no_duplicates = df_school_geoloc.drop_duplicates(subset='Name')

# Display the DataFrame after removing duplicates
display(df_school_geoloc_no_duplicates)

,Unnamed: 0,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,Name
0,13,BLANGAH RISE PRIMARY SCHOOL,91,TELOK BLANGAH HEIGHTS,BLANGAH RISE PRIMARY SCHOOL,91 TELOK BLANGAH HEIGHTS BLANGAH RISE PRIMARY ...,109100,25252.19390,28732.45261,1.276120,103.808628,Blangah Rise Primary School
1,14,SHG STUDENT CARE (BLANGAH RISE PRIMARY SCHOOL),91,TELOK BLANGAH HEIGHTS,SHG STUDENT CARE (BLANGAH RISE PRIMARY SCHOOL),91 TELOK BLANGAH HEIGHTS SHG STUDENT CARE (BLA...,109100,25250.48345,28735.18935,1.276145,103.808613,Blangah Rise Primary School
2,15,HOLY INNOCENTS' PRIMARY SCHOOL,5,LORONG LOW KOON,HOLY INNOCENTS' PRIMARY SCHOOL,5 LORONG LOW KOON HOLY INNOCENTS' PRIMARY SCHO...,536451,34765.90368,38774.69439,1.366938,103.894115,Holy Innocents' Primary School
3,16,JURONG WEST PRIMARY SCHOOL,30,JURONG WEST STREET 61,JURONG WEST PRIMARY SCHOOL,30 JURONG WEST STREET 61 JURONG WEST PRIMARY S...,648368,13029.92453,35705.11652,1.339175,103.698804,Jurong West Primary School
4,18,KEMING PRIMARY SCHOOL,90,BUKIT BATOK EAST AVENUE 6,KEMING PRIMARY SCHOOL,90 BUKIT BATOK EAST AVENUE 6 KEMING PRIMARY SC...,659762,19445.30317,36407.35614,1.345528,103.756449,Keming Primary School
...,...,...,...,...,...,...,...,...,...,...,...,...
252,437,BIG HEART STUDENT CARE (PRINCESS ELIZABETH PRI...,30,BUKIT BATOK WEST AVENUE 3,BIG HEART STUDENT CARE (PRINCESS ELIZABETH PRI...,30 BUKIT BATOK WEST AVENUE 3 BIG HEART STUDENT...,659163,17694.71934,36812.39679,1.349191,103.740719,Princess Elizabeth Primary School
253,438,PRINCESS ELIZABETH PRIMARY SCHOOL,30,BUKIT BATOK WEST AVENUE 3,PRINCESS ELIZABETH PRIMARY SCHOOL,30 BUKIT BATOK WEST AVENUE 3 PRINCESS ELIZABET...,659163,17728.15468,36813.78037,1.349204,103.741020,Princess Elizabeth Primary School
254,443,FORMER TAO NAN SCHOOL,39,ARMENIAN STREET,FORMER TAO NAN SCHOOL,39 ARMENIAN STREET FORMER TAO NAN SCHOOL SINGA...,179941,29748.11651,30746.19234,1.294332,103.849026,Tao Nan School
255,444,TAO NAN SCHOOL,49,MARINE CRESCENT,TAO NAN SCHOOL,49 MARINE CRESCENT TAO NAN SCHOOL SINGAPORE 44...,449761,36706.79080,31957.46868,1.305285,103.911553,Tao Nan School


,Unnamed: 0,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,Name
0,13,BLANGAH RISE PRIMARY SCHOOL,91,TELOK BLANGAH HEIGHTS,BLANGAH RISE PRIMARY SCHOOL,91 TELOK BLANGAH HEIGHTS BLANGAH RISE PRIMARY ...,109100,25252.19390,28732.45261,1.276120,103.808628,Blangah Rise Primary School
1,14,SHG STUDENT CARE (BLANGAH RISE PRIMARY SCHOOL),91,TELOK BLANGAH HEIGHTS,SHG STUDENT CARE (BLANGAH RISE PRIMARY SCHOOL),91 TELOK BLANGAH HEIGHTS SHG STUDENT CARE (BLA...,109100,25250.48345,28735.18935,1.276145,103.808613,Blangah Rise Primary School
2,15,HOLY INNOCENTS' PRIMARY SCHOOL,5,LORONG LOW KOON,HOLY INNOCENTS' PRIMARY SCHOOL,5 LORONG LOW KOON HOLY INNOCENTS' PRIMARY SCHO...,536451,34765.90368,38774.69439,1.366938,103.894115,Holy Innocents' Primary School
3,16,JURONG WEST PRIMARY SCHOOL,30,JURONG WEST STREET 61,JURONG WEST PRIMARY SCHOOL,30 JURONG WEST STREET 61 JURONG WEST PRIMARY S...,648368,13029.92453,35705.11652,1.339175,103.698804,Jurong West Primary School
4,18,KEMING PRIMARY SCHOOL,90,BUKIT BATOK EAST AVENUE 6,KEMING PRIMARY SCHOOL,90 BUKIT BATOK EAST AVENUE 6 KEMING PRIMARY SC...,659762,19445.30317,36407.35614,1.345528,103.756449,Keming Primary School
...,...,...,...,...,...,...,...,...,...,...,...,...
252,437,BIG HEART STUDENT CARE (PRINCESS ELIZABETH PRI...,30,BUKIT BATOK WEST AVENUE 3,BIG HEART STUDENT CARE (PRINCESS ELIZABETH PRI...,30 BUKIT BATOK WEST AVENUE 3 BIG HEART STUDENT...,659163,17694.71934,36812.39679,1.349191,103.740719,Princess Elizabeth Primary School
253,438,PRINCESS ELIZABETH PRIMARY SCHOOL,30,BUKIT BATOK WEST AVENUE 3,PRINCESS ELIZABETH PRIMARY SCHOOL,30 BUKIT BATOK WEST AVENUE 3 PRINCESS ELIZABET...,659163,17728.15468,36813.78037,1.349204,103.741020,Princess Elizabeth Primary School
254,443,FORMER TAO NAN SCHOOL,39,ARMENIAN STREET,FORMER TAO NAN SCHOOL,39 ARMENIAN STREET FORMER TAO NAN SCHOOL SINGA...,179941,29748.11651,30746.19234,1.294332,103.849026,Tao Nan School
255,444,TAO NAN SCHOOL,49,MARINE CRESCENT,TAO NAN SCHOOL,49 MARINE CRESCENT TAO NAN SCHOOL SINGAPORE 44...,449761,36706.79080,31957.46868,1.305285,103.911553,Tao Nan School


,Unnamed: 0,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,Name
0,13,BLANGAH RISE PRIMARY SCHOOL,91,TELOK BLANGAH HEIGHTS,BLANGAH RISE PRIMARY SCHOOL,91 TELOK BLANGAH HEIGHTS BLANGAH RISE PRIMARY ...,109100,25252.19390,28732.45261,1.276120,103.808628,Blangah Rise Primary School
2,15,HOLY INNOCENTS' PRIMARY SCHOOL,5,LORONG LOW KOON,HOLY INNOCENTS' PRIMARY SCHOOL,5 LORONG LOW KOON HOLY INNOCENTS' PRIMARY SCHO...,536451,34765.90368,38774.69439,1.366938,103.894115,Holy Innocents' Primary School
3,16,JURONG WEST PRIMARY SCHOOL,30,JURONG WEST STREET 61,JURONG WEST PRIMARY SCHOOL,30 JURONG WEST STREET 61 JURONG WEST PRIMARY S...,648368,13029.92453,35705.11652,1.339175,103.698804,Jurong West Primary School
4,18,KEMING PRIMARY SCHOOL,90,BUKIT BATOK EAST AVENUE 6,KEMING PRIMARY SCHOOL,90 BUKIT BATOK EAST AVENUE 6 KEMING PRIMARY SC...,659762,19445.30317,36407.35614,1.345528,103.756449,Keming Primary School
6,28,ANGLO-CHINESE SCHOOL (PRIMARY),50,BARKER ROAD,ANGLO-CHINESE SCHOOL (PRIMARY),50 BARKER ROAD ANGLO-CHINESE SCHOOL (PRIMARY) ...,309918,28254.98295,33404.23402,1.318371,103.835610,Anglo-Chinese School (Primary)
...,...,...,...,...,...,...,...,...,...,...,...,...
249,434,WOODGROVE PRIMARY SCHOOL,2,WOODLANDS DRIVE 14,WOODGROVE PRIMARY SCHOOL,2 WOODLANDS DRIVE 14 WOODGROVE PRIMARY SCHOOL ...,738079,23246.56051,46090.85424,1.433103,103.790604,Woodgrove Primary School
251,436,ZHENGHUA PRIMARY SCHOOL,9,FAJAR ROAD,ZHENGHUA PRIMARY SCHOOL,9 FAJAR ROAD ZHENGHUA PRIMARY SCHOOL SINGAPORE...,679002,20877.03954,40169.11691,1.379549,103.769313,Zhenghua Primary School
252,437,BIG HEART STUDENT CARE (PRINCESS ELIZABETH PRI...,30,BUKIT BATOK WEST AVENUE 3,BIG HEART STUDENT CARE (PRINCESS ELIZABETH PRI...,30 BUKIT BATOK WEST AVENUE 3 BIG HEART STUDENT...,659163,17694.71934,36812.39679,1.349191,103.740719,Princess Elizabeth Primary School
254,443,FORMER TAO NAN SCHOOL,39,ARMENIAN STREET,FORMER TAO NAN SCHOOL,39 ARMENIAN STREET FORMER TAO NAN SCHOOL SINGA...,179941,29748.11651,30746.19234,1.294332,103.849026,Tao Nan School


In [23]:
school_postal1 = df_school_geoloc_no_duplicates[['Name', 'LATITUDE', 'LONGITUDE']]
display(school_postal1)     

map_school_postal = find_nearest(resale_postal1 , school_postal1)
save_file(map_school_postal,"map_school_updated")


,Name,LATITUDE,LONGITUDE
0,Blangah Rise Primary School,1.276120,103.808628
2,Holy Innocents' Primary School,1.366938,103.894115
3,Jurong West Primary School,1.339175,103.698804
4,Keming Primary School,1.345528,103.756449
6,Anglo-Chinese School (Primary),1.318371,103.835610
...,...,...,...
249,Woodgrove Primary School,1.433103,103.790604
251,Zhenghua Primary School,1.379549,103.769313
252,Princess Elizabeth Primary School,1.349191,103.740719
254,Tao Nan School,1.294332,103.849026


Processed 1000/155821 houses
Processed 2000/155821 houses
Processed 3000/155821 houses
Processed 4000/155821 houses
Processed 5000/155821 houses
Processed 6000/155821 houses
Processed 7000/155821 houses
Processed 8000/155821 houses
Processed 9000/155821 houses
Processed 10000/155821 houses
Processed 11000/155821 houses
Processed 12000/155821 houses
Processed 13000/155821 houses
Processed 14000/155821 houses
Processed 15000/155821 houses
Processed 16000/155821 houses
Processed 17000/155821 houses
Processed 18000/155821 houses
Processed 19000/155821 houses
Processed 20000/155821 houses
Processed 21000/155821 houses
Processed 22000/155821 houses
Processed 23000/155821 houses
Processed 24000/155821 houses
Processed 25000/155821 houses
Processed 26000/155821 houses
Processed 27000/155821 houses
Processed 28000/155821 houses
Processed 29000/155821 houses
Processed 30000/155821 houses
Processed 31000/155821 houses
Processed 32000/155821 houses
Processed 33000/155821 houses
Processed 34000/155

In [24]:
map_school_updated = pd.read_csv('map_school_updated.csv')
display(map_school_updated)


,0,1,2
0,406 ANG MO KIO AVE 10,Townsville Primary School,0.2181254395256675 km
1,108 ANG MO KIO AVE 4,Ang Mo Kio Primary School,0.2415753425097763 km
2,602 ANG MO KIO AVE 5,Anderson Primary School,0.7771571662417865 km
3,465 ANG MO KIO AVE 10,Teck Ghee Primary School,0.6981684698310231 km
4,601 ANG MO KIO AVE 5,Anderson Primary School,0.7825550706448622 km
...,...,...,...
9455,363B SEMBAWANG CRES,Canberra Primary School,0.6558821907896819 km
9456,365C SEMBAWANG CRES,Sembawang Primary School,0.4879646670499008 km
9457,362A SEMBAWANG CRES,Canberra Primary School,0.4934690117872322 km
9458,366B SEMBAWANG CRES,Sembawang Primary School,0.5800262185152377 km


In [26]:

# read hdb property info dataset
# HDB property information contains the location of existing HDB blocks, 
#highest floor level, year of completion, type of building and number of HDB flats (breakdown by flat type) per block etc.

df_property_info = pd.read_csv('hdb-property-information.csv')
df_property_info = df_property_info[['blk_no', 'street', 'max_floor_lvl', 'year_completed', 
                  'market_hawker', 'multistorey_carpark', 'total_dwelling_units']]

In [27]:
# update df_resale
df_resale = pd.read_csv('resale_flat_with_postal.csv')

In [28]:
# Prepare data for merging 
df_school = pd.read_csv('map_school_updated.csv')
df_school.columns = ['address','nearest_school', 'distance_from_school']

In [29]:
# Merge datasets
resale_property_info = pd.merge(df_resale, df_property_info, how='left', 
               left_on=['block', 'street_name'], right_on=['blk_no', 'street'])
resale_and_school1 = pd.merge(df_resale, df_school, how='left', left_on=['address_clean'], right_on=['address'])

# clean after merge
merged_school = resale_and_school1[:]

merged_school['flat_remainlease'] =  merged_school['remaining_lease'].str[:2].astype(int)
merged_school['est_floor'] =  merged_school['storey_range'].str[:2].astype(int)
merged_school['distance_school'] =  merged_school['distance_from_school'].str[:-2].astype(float)

merged_school['school_distribution'] =  np.where(merged_school['distance_school'] < 1, 'within-1km', 'More than-1km')

# export 
merged_school.to_csv ("final_data_with_school.csv",index = None, header=True)

df_final = pd.read_csv("final_data_with_school.csv")
print(df_final.info())
df_final.head(3)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2380\2598651787.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_school['flat_remainlease'] =  merged_school['remaining_lease'].str[:2].astype(int)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2380\2598651787.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_school['est_floor'] =  merged_school['storey_range'].str[:2].astype(int)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2380\2598651787.py:11: SettingWithCopyWarning: 
A value is trying to be s

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155821 entries, 0 to 155820
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   month                           155821 non-null  object 
 1   town                            155821 non-null  object 
 2   public_housing_flat_type        155821 non-null  object 
 3   block                           155821 non-null  object 
 4   street_name                     155821 non-null  object 
 5   storey_range                    155821 non-null  object 
 6   floor_area_sqm                  155821 non-null  float64
 7   flat_model                      155821 non-null  object 
 8   lease_commence_date             155821 non-null  int64  
 9   remaining_lease                 155821 non-null  object 
 10  resale_price_Singapore_dollars  155821 non-null  float64
 11  Year                            155821 non-null  int64  
 12  mth             

,month,town,public_housing_flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,LATITUDE,LONGITUDE,POSTAL,address,nearest_school,distance_from_school,flat_remainlease,est_floor,distance_school,school_distribution
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,...,1.362005,103.853880,560406.0,406 ANG MO KIO AVE 10,Townsville Primary School,0.2181254395256675 km,61,10,0.218125,within-1km
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,...,1.370966,103.838202,560108.0,108 ANG MO KIO AVE 4,Ang Mo Kio Primary School,0.2415753425097763 km,60,1,0.241575,within-1km
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,...,1.380709,103.835368,560602.0,602 ANG MO KIO AVE 5,Anderson Primary School,0.7771571662417865 km,62,1,0.777157,within-1km
